# Chain

**Chain**(체인)은 여러 컴포넌트(요소)를 정해진 순서대로 연결하여 **복잡한 AI 작업을 단계별로 자동화**할 수 있도록 돕는 구조이다.

- 각 컴포넌트는 입력을 받아 특정 처리를 수행한 후 다음 단계로 결과를 전달한다.
- 복잡한 작업을 여러 개의 단순한 단계로 나누고, 각 단계를 순차적으로 실행함으로써 전체 작업을 체계적으로 구성할 수 있다.

## 기본 개념

- 체인은 하나의 LLM 호출에 그치지 않고 **여러 LLM 호출이나 도구 실행을 순차적으로 연결**할 수 있다.
- 예를 들어, 사용자의 질문 → 검색 → 요약 → 응답 생성 같은 일련의 작업을 체인으로 구성할 수 있다.
- 체인을 사용하면 코드의 재사용성과 유지 보수성이 향상된다.

## LangChain에서의 Chain 구성 방식

LangChain은 다음 두 가지 방식을 통해 체인을 구성할 수 있다.

### 1. Off-the-shelf Chains 방식 (클래식 방식)

- LangChain에서 제공하는 **미리 정의된 Chain 클래스**(예: `LLMChain`, `SequentialChain`, `SimpleSequentialChain`)를 활용하는 방식이다.
- 이 방식은 LangChain의 **초기 구조**이며, 대부분의 클래스는 현재 **더 이상 사용되지 않음(deprecated)** 상태이다.

> 현재 LangChain에서는 이 방식을 권장하지 않는다.

### 2. LCEL (LangChain Expression Language) 방식

- 체인을 함수형 방식으로 선언할 수 있는 **표현식 기반의 체인 구성 언어**이다.
- LCEL 방식은 간결하고 선언적인 문법을 제공하여 **직관적이고 확장성 있는 체인 구성**이 가능하다.
- `Runnable`이라는 공통 인터페이스를 기반으로 다양한 요소를 조합하여 체인을 구성한다.
- 체인의 각 구성 요소는 `invoke()` 메서드로 실행된다.

In [1]:
# off-the-shelf 방식
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

prompt_template = PromptTemplate(
    template="{item}에 어울리는 이름 {count}개를 만들어 주세요."
)
model = ChatOpenAI(model_name="gpt-4o-mini")
parser = StrOutputParser()

# 변수:값 -> (prompt_template) -prompt-> (model) -응답결과-> (parser) -> 최종결과

In [2]:
from langchain import LLMChain
chain = LLMChain(
    prompt=prompt_template,
    llm=model, 
    output_parser=parser
)
response = chain.invoke({"item":"가방", "count":5})

C:\Users\Playdata\AppData\Local\Temp\ipykernel_11036\1686607166.py:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(


In [3]:
response

{'item': '가방',
 'count': 5,
 'text': '물론입니다! 가방에 어울리는 이름 5개를 제안해 드릴게요.\n\n1. **에코백** - 자연 친화적인 스타일을 강조한 이름\n2. **모던 클러치** - 세련되고 현대적인 느낌을 주는 이름\n3. **트래블 메이트** - 여행과 함께하는 가방이라는 의미를 담은 이름\n4. **스타일리시 캐리어** - 멋과 실용성을 동시에 표현한 이름\n5. **러닝 투고** - 활동적인 라이프스타일에 어울리는 이름\n\n이름이 마음에 드시길 바랍니다!'}

In [4]:
print(response["text"])

물론입니다! 가방에 어울리는 이름 5개를 제안해 드릴게요.

1. **에코백** - 자연 친화적인 스타일을 강조한 이름
2. **모던 클러치** - 세련되고 현대적인 느낌을 주는 이름
3. **트래블 메이트** - 여행과 함께하는 가방이라는 의미를 담은 이름
4. **스타일리시 캐리어** - 멋과 실용성을 동시에 표현한 이름
5. **러닝 투고** - 활동적인 라이프스타일에 어울리는 이름

이름이 마음에 드시길 바랍니다!


# [LCEL](https://python.langchain.com/docs/how_to/#langchain-expression-language-lcel) (LangChain Expression Language)
- LCEL은 LangChain의 핵심 기능인 체인(Chain)을 더욱 간결하고 유연하게 구성할 수 있도록 고안된 **선언형 체인(chain) 구성 언어**이다.
- 파이프 연산자 `|`를 사용해 선언적 방법으로 여러 작업을 연결한다.
- 체인을 구성하는 각 요소는 `Runnable` 타입으로, 체인 내에서 실행 가능한 단위이다.
- 각 단계는 invoke() 메서드를 통해 실행되며, 앞 단계의 출력이 다음 단계의 입력으로 자동 전달된다.
    - [Runnable 컴포넌트별 입출력 타입](https://python.langchain.com/docs/concepts/runnables/#input-and-output-types)
    - 각 컴포넌트의 input과 output 타입에 맞춰 값이 전달되도록 한다.
- https://python.langchain.com/v0.2/docs/concepts/#langchain-expression-language-lcel

## [Runnable](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.base.Runnable.html)
- LangChain의 Runnable은 실행 가능한 작업 단위를 캡슐화한 개념으로, 데이터 흐름의 각 단계를 정의하고 **체인(chain) 에 포함 되어**  복잡한 작업의 각 단계를 수행 한다.
- Chain을 구성하는 class들은 Runnable의 상속 받아 구현한다.
- **Prompt Template클래스**, **Chat 모델, LLM 모델 클래스**, **Output Parser 클래스** 등 다양한 컴포넌트가 Runnable을 상속받아 구현된다.

### 주요 특징
- 작업 단위의 캡슐화:
    - Runnable은 특정 작업(예: 프롬프트 생성, LLM 호출, 출력 파싱 등)을 수행하는 독립적인 컴포넌트이다.
    - 각 컴포넌트는 독립적으로 테스트 및 재사용이 가능하며, 조합하여 복잡한 체인을 구성할 수 있다.
- 체인 연결 및 작업 흐름 관리:
    - Runnable은 체인(chain, 일련의 연결된 작업 흐름)을 구성하는 기본 단위로 사용된다.
    - LangChain Expression Language(LCEL)를 사용하면 | 연산자를 통해 여러 Runnable을 쉽게 연결할 수 있다.
    - 입력과 출력의 형식을 일관되게 유지하여 각 단계가 자연스럽게 연결된다.
- 모듈화 및 디버깅 용이성:
    - 각 단계가 명확히 분리되어 문제 발생 시 어느 단계에서 오류가 발생했는지 쉽게 확인할 수 있다.
    - 복잡한 작업을 작은 단위로 나누어 체계적으로 관리할 수 있다.
      
### Runnable의 표준 메소드
- 모든 Runnable이 구현하는 공통 메소드
    - `invoke()`: 단일 입력을 처리하여 결과를 반환.
    - `batch()`: 여러 입력 데이터들을 한 번에 처리.
    - `stream()`: 입력에 대해 스트리밍 방식으로 응답을 반환.
    - `ainvoke()`: 비동기 방식으로 입력을 처리하여 결과를 반환.

### Runnable의 주요 구현체(하위 클래스)

- `RunnableSequence`
    - 여러 `Runnable`을 순차적으로 연결하여 실행하는 구성이다.
    - 각 단계의 출력이 다음 단계의 입력으로 전달된다.
    - LCEL을 사용하여 체인을 구성할 경우 자동으로 `RunnableSequence`로 변환된다.
-  `RunnablePassThrough`
    - 입력 데이터를 가공하지 않고 그대로 다음 단계로 전달하는 `Runnable`이다.
    - 선택적으로 미리 정의된 키-값 쌍을 함께 전달할 수 있다.

- `RunnableParallel`
    - 여러 `Runnable`을 병렬로 실행한 후, 결과를 결합하여 다음 단계로 전달한다.
    - 병렬 처리를 통해 처리 속도를 개선할 수 있다.

- `RunnableLambda`
    - 일반 함수 또는 `lambda` 함수를 `Runnable`로 변환하여 체인에 포함할 수 있다.
    - 사용자 정의 함수로 동작을 확장할 때 유용하다.

In [6]:
from dotenv import load_dotenv
load_dotenv()

True

#### Runnable 예제

In [ ]:
from langchain_core.runnables import Runnable  # 모든 Runnable의 최상위.
# 사용자정의 Runnable
class MyRunnable(Runnable):

    def invoke(self, input_data:str, config:dict=None):
        # invoke(): 구현하는 Runnable이 해야하는 작업을 구현하는 메소드.
        # input_data: 입력값
        # config: 일할 때 필요한 설정값들.
        if config is not None and config.get('lang') == "en":
            return f"Explain {input_data} in one sentences."
        return f"{input_data}에 대해서 한 문장으로 설명해줘."

In [13]:
my_runnable = MyRunnable()
my_runnable.invoke("사과")
my_runnable.invoke("컴퓨터")
my_runnable.invoke("Apple", {"lang":"en"})

'Explain Apple in one sentences.'

In [15]:
from langchain_openai import ChatOpenAI

my_runnable = MyRunnable()
model = ChatOpenAI(model_name="gpt-4o-mini")

prompt = my_runnable.invoke("Apple", {"lang":"en"})
prompt = my_runnable.invoke("Langchain")
response = model.invoke(prompt)
print(response)

content='Langchain은 다양한 언어 모델을 연결하고 활용하여 복잡한 애플리케이션을 구축하는 데 도움을 주는 개발 프레임워크입니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 19, 'total_tokens': 54, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-BgjIzSM7rZgIo7KuycyA1Kz9XxhNR', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--7ff04cc7-a563-40b7-a78a-9a09822de099-0' usage_metadata={'input_tokens': 19, 'output_tokens': 35, 'total_tokens': 54, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [16]:
# chain -> Runnable | Runnable | Runnable
chain = my_runnable | model
# chain 호출 -> invoke
res = chain.invoke("과일 배")
print(res)

content='과일 배는 달콤하고 즙이 많은 속과 부드러운 껍질을 가진 과일로, 주로 생으로 섭취하거나 요리와 디저트에 사용됩니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 20, 'total_tokens': 65, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-BgjLYzwH8fpTLnnMO1NI5DYLTYKAB', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--49b62362-d46d-4ad6-82e9-203286a93874-0' usage_metadata={'input_tokens': 20, 'output_tokens': 45, 'total_tokens': 65, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [ ]:
# 기본 체인 구성: prompt_template -> model -> output parser
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, CommaSeparatedListOutputParser

# role: system, user/human, ai/assistant
#       system: 채팅  전체에 적용되는 공통 지침을 지정하는 role
prompt_template = ChatPromptTemplate(
    messages=[
        ("system", "당신은 오랜 경력의 한국 관광 가이드입니다. 여행객들에게 설명하듯이 친절하게 답변을 해주세요."),
        ("human", "{query}")
    ]
)
model = ChatOpenAI(model_name="gpt-4o-mini", temperature=1.0)

guide_chain = prompt_template | model | StrOutputParser()

print(type(guide_chain)) # RunnableSequence: Runnable 타입 
#                           ==> chain도 다른 chain의 구성요소로 포함될 수있다.

<class 'langchain_core.runnables.base.RunnableSequence'>


In [20]:
query = "서울에서 꼭 가봐야되는 여행지를 세 곳만 알려줘."
response = guide_chain.invoke({"query":query})

In [21]:
print(response)

서울에는 정말 많은 매력적인 여행지가 있지만, 그중에서도 꼭 가봐야 할 세 곳을 소개해드릴게요.

1. **경복궁**: 경복궁은 조선 왕조의 가장 큰 궁궐로, 한국의 전통 건축을 느낄 수 있는 멋진 곳입니다. 고궁 내에는 국립고궁박물관도 있어, 한국의 역사와 문화를 더 깊이 이해할 수 있습니다. 매일 정해진 시간에 열리는 수문장 교대식도 놓치지 말아야 할 볼거리입니다.

2. **남산서울타워**: 남산의 꼭대기에 위치한 서울타워는 서울을 한눈에 내려다볼 수 있는 멋진 전망대입니다. 특히 일몰 시간대에 가면 더욱 아름다운 경치를 감상할 수 있는데요, 타워 주변에 조성된 공원과 케이블카도 함께 즐길 수 있어 가족 단위 여행객에게도 추천합니다.

3. **명동**: 서울의 대표적인 쇼핑 거리인 명동은 기념품 쇼핑이나 길거리 음식을 즐기기에 최적의 장소입니다. 다양한 브랜드와 로컬 상점들이 밀집해 있어 shopping을 즐길 수 있고, 유명한 명동 떡볶이와 같은 길거리 음식을 꼭 드셔보세요! 또한, 명동 성당도 인상적인 건축물로, 여유로운 산책을 즐기기에 좋답니다.

이 세 곳은 서울의 역사, 문화, 현대적 매력을 한꺼번에 느낄 수 있는 장소들입니다. 서울 여행을 하시면서 꼭 들러보세요!


In [23]:
while True:
    query = input("질문:")
    if query == "!quit":
        break
    resp = guide_chain.invoke({"query":query})
    print("User:", query)
    print("AI:", resp)
    print("-"* 50)

User: 경복궁에 대해서 설명해줘.
AI: 안녕하세요! 경복궁에 오신 것을 환영합니다. 경복궁은 대한민국 서울특별시에 위치한 조선 왕조의 대표적인 궁궐 중 하나로, 1395년에 지어졌습니다. 이곳은 조선의 왕들이 살았던 곳으로, 풍부한 역사와 아름다운 건축물로 유명합니다.

경복궁의 이름은 "가득 차고 복이 온다"는 뜻을 가지고 있는데요, 이것은 조선 왕조가 국가의繁栄과 평화를 기원하던 의미를 담고 있습니다. 특히 이 궁궐은 조선 시대의 궁궐 중 가장 크고 화려한 건축물로, 대표적인 건축물로는 근정전, 경회루, 사정전을 들 수 있습니다.

**근정전**은 조선의 왕들이 즉위식이나 중요 의식을 치르던 곳으로, 대규모 행사와 국정을 다루는 장소였습니다. 이곳의 아름다운 외관과 화려한 기둥 위에 얹혀 있는 지붕은 경복궁의 상징이라고 할 수 있습니다.

**경회루**는 궁궐의 연못에 세워진 정자로, 조선 왕들이 연회를 열거나 외로운 저녁을 즐기는 장소였죠. 경회루에 서면 주변의 아름다운 조경과 함께 한 폭의 그림 같은 풍경을 감상할 수 있습니다.

여름철에 인근에서 열리는 전통 공연도 놓치지 말아야 할 관람 포인트입니다. 궁궐 내에서는 매일 정오에 진행되는 교대식도 관람할 수 있으며, 이는 한국의 전통적인 문화를 경험하기 좋은 기회입니다.

경복궁은 단순한 궁궐 이상의 의미를 지니고 있으며, 한국의 역사와 문화가 숨쉬는 장소입니다. 방문하시면 조선 왕조의 과거를 느끼는 특별한 경험이 될 것입니다. 궁궐 내 산책을 즐기시면서 아름다운 경치를 만끽해 보세요!
--------------------------------------------------
User: ?
AI: 안녕하세요! 무엇을 도와드릴까요? 한국 여행에 대해 궁금한 점이나 계획하고 있는 관광지에 대해 알려주시면 친절하게 설명해 드리겠습니다. 편하게 질문해 주세요!
--------------------------------------------------
User: 명동에 가면 꼭 가봐야 할 곳은?
AI: 명

#### RunnableLambda 예제

In [25]:
from langchain_core.runnables import RunnableLambda
# RunnableLambda(함수) -> 함수를 실행하는 Runnable을 생성.
my_runnable2 = RunnableLambda(lambda input_data : f"{input_data}를 한 문장으로 설명해줘.")
my_runnable2.invoke("LLM") 

'LLM를 한 문장으로 설명해줘.'

In [26]:
chain = my_runnable2 | model
chain.invoke("LLM")

AIMessage(content='LLM(대형 언어 모델)은 방대한 양의 텍스트 데이터를 기반으로 언어를 이해하고 생성할 수 있도록 훈련된 인공지능 모델입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 18, 'total_tokens': 56, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BgkAAty6ZVMd6IcUMSqaPIfSyDPhM', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--f65ccf45-541e-433e-9476-cc9348391ef4-0', usage_metadata={'input_tokens': 18, 'output_tokens': 38, 'total_tokens': 56, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
def sum(nums:dict):
    return nums[0] + nums[1]

my_runnable2 = RunnableLambda(sum)
my_runnable2.invoke({0:10, 1:20})

# invoke(입력데이터:str|dict, 설정정보:dict)
## 입력데이터가 여러개일 경우 dict등의 자료구조를 이용해서 받는다.

30

#### RunnablePassthrough 예제

In [31]:
# 1 앞 Runnable이 처리한 결과를 다음 Runnable에 그대로 전달.
from langchain_core.runnables import RunnablePassthrough

RunnablePassthrough().invoke("안녕하세요")
RunnablePassthrough().invoke({"Key":"value"})

{'Key': 'value'}

In [32]:
# 2. 앞 Runnable이 처리한 결과에 Item을 추가해서 다음 Runnable에 전달.
#   -> 입력으로 dictionary 받아서 거기에 item을 추가.
# RunnablePassthrough.assign(key1=Runnable, key2=Runnable, ...)
#  - 받은 dictionary에 "key1":Runnable반환값, "key2":Runnable반환값, .. 추가해서 다음으로 전달.
address_runnable = RunnableLambda(lambda x: "서울시 금천구")  #"서울시 금천구" 를 반환.
phone_runnable = RunnableLambda(lambda x: "010-1111-2222")

RunnablePassthrough.assign(address=address_runnable, phone=phone_runnable).invoke({"name":"홍길동"})

{'name': '홍길동', 'address': '서울시 금천구', 'phone': '010-1111-2222'}

#### RunnableSequence 예제

In [34]:
from langchain_core.runnables import RunnableSequence

run1 = RunnableLambda(lambda x: x + 1)
run2 = RunnableLambda(lambda x: x * 2)

chain = run1 | run2
print(type(chain))
chain.invoke(30)


<class 'langchain_core.runnables.base.RunnableSequence'>


62

In [35]:
chain2 = RunnableSequence(run1, run2)  #(prompt_template, model,  output_parser)
chain2.invoke(100)

202

#### RunnableParallel 예제

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnableLambda

run1 = RunnableLambda(lambda x: x + 1)
run2 = RunnableLambda(lambda x: x * 2)
run3 = RunnableLambda(lambda x: x // 3)

runnable = RunnableParallel(
    {
        "result1":run1,
        "result2":run2,
        "result3":run3,
        "result4":RunnablePassthrough()  # 앞에서 받은 값을 그대로 다음에 전달.
    }
)
# Runnable들을 각각 실행하고 그 결과를 key에 할당한 Dictionary를 반환.
runnable.invoke(20)

{'result1': 21, 'result2': 40, 'result3': 6, 'result4': 20}

#### LCEL Chain 예제

In [61]:
# 음식 이름을 받아서 레시피를 "영어"로 출력하는 chain을 구성
# prompt template -> model -> output parser 
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from textwrap import dedent
prompt_template = PromptTemplate(
    template=dedent("""
    # Instruction
    당신은 숙련된 요리 연구가입니다. 요청한 음식의 레시피를 작성해 주세요.
                    
    # Input data
    음식이름: {food}
    """)
)
model = ChatOpenAI(model_name="gpt-4o-mini")
food_chain = prompt_template | model | StrOutputParser()


In [62]:
response = food_chain.invoke({"food":"pasta"})

In [63]:
print(response)

## 파스타 레시피

### 재료
- 파스타 (스파게티, 페투치니 등) 200g
- 물 2리터
- 소금 10g
- 올리브 오일 2큰술
- 마늘 2쪽 (다진 것)
- 양파 1개 (다진 것)
- 토마토 소스 400g (또는 신선한 토마토 4개)
- 바질 잎 (신선한 또는 말린 것)
- 파마산 치즈 (갈아놓은 것) 적당량
- 후추, 소금 (간용)

### 조리 방법

1. **파스타 삶기**:
   - 큰 냄비에 물 2리터를 붓고 끓입니다.
   - 물이 끓기 시작하면 소금을 넣습니다.
   - 파스타를 넣고 포장에 적힌 시간동안 삶습니다. (보통 8-12분)
   - 삶아진 파스타는 체에 걸러 물기를 제거합니다. 

2. **소스 만들기**:
   - 팬에 올리브 오일을 중불로 가열합니다.
   - 다진 마늘과 양파를 넣고 볶아 양파가 투명해질 때까지 익힙니다.
   - 토마토 소스를 추가하고 잘 섞은 후, 약한 불로 10-15분 정도 조리하여 소스가 농축되게 합니다. (신선한 토마토를 사용하는 경우, 먼저 익혀서 주스와 피부를 제거해줍니다.)
   - 바질 잎을 넣고 간을 맞추기 위해 소금과 후추를 추가합니다.

3. **파스타와 소스 혼합**:
   - 삶아놓은 파스타를 소스가 있는 팬에 넣고 잘 섞어서 1-2분간 조리합니다. 이때 파스타가 소스를 잘 흡수하도록 합니다.

4. **서빙**:
   - 파스타를 접시에 담고, 갈아놓은 파마산 치즈를 뿌린 후, 원하신다면 추가적인 바질 잎으로 장식합니다.
   - 뜨겁게 서빙합니다.

### 팁
- 다양한 재료 (채소, 해산물, 고기 등)를 추가하여 취향에 맞는 변형을 시도해보세요.
- 알리오 올리오 스타일이나 크림 소스를 사용하여 다른 맛을 즐길 수도 있습니다.

맛있게 드세요!


In [ ]:
# 번역 chain -> 입력된 내용을 지정한(입력한) 언어로 번역하는 체인.
# prompt template -> model -> output parser 

prompt_template_trans = PromptTemplate(
    template=dedent("""
    # Instruction
    당신은 다국어가능한 숙력된 번역가입니다. 
    요청된 문장을 {language} 로 번역해 주세요.
                    
    # Input data(번역할 문장)
    {content}
                    
    # Output Indicator
    - 번역한 내용만 출력해 주세요.
    """)
)

translate_chain = prompt_template_trans | model | StrOutputParser()


In [ ]:
translate_chain.invoke({"content":"안녕하세요", "language":"독일어"})
# 안녕하세요->독일어로 번역한 응답.



'Hallo'

In [55]:
response_kor = translate_chain.invoke({"content":response, "language":"한국어"})

In [56]:
print(response_kor)

물론입니다! 클래식한 파스타 요리인 스파게티 알리오 올리오(마늘과 올리브 오일을 곁들인 스파게티)의 간단한 레시피를 소개합니다.

### 스파게티 알리오 올리오 레시피

#### 재료:

- 스파게티 400g
- 마늘 6쪽, 얇게 슬라이스
- 엑스트라 버진 올리브 오일 1/2컵
- 붉은 고추 flakes 1/4티스푼 (입맛에 맞게 조절)
- 소금 (맛에 따라)
- 갓 갈은 후추 (맛에 따라)
- 신선한 파슬리 1/4컵, 다진 것
- 파르마산 치즈 (선택 사항, 서빙용)
- 레몬 제스트 (선택 사항, 서빙용)

#### 조리법:

1. **파스타 삶기**:
   - 큰 냄비에 소금을 넣은 물을 끓입니다. 스파게티를 추가하고 패키지 지침에 따라 알던테가 될 때까지 삶습니다. 약 1컵의 파스타 물을 남겨두고 파스타를 Drain합니다.

2. **마늘과 오일 준비하기**:
   - 파스타를 삶는 동안 큰 팬에 중불에서 올리브 오일을 가열합니다.
   - 슬라이스한 마늘과 붉은 고추 flakes를 넣습니다. 마늘이 황금빛 갈색이 되고 향이 날 때까지 자주 저어가며 3-4분간 요리합니다. 마늘을 태우지 않도록 주의하세요. 태우면 쓴맛이 날 수 있습니다.

3. **파스타와 소스 결합**:
   - Drain한 스파게티를 마늘과 오일이 들어 있는 팬에 추가합니다. 파스타가 고르게 오일에 묻도록 잘 섞어줍니다.
   - 만약 파스타가 건조해 보인다면, 남겨두었던 파스타 물을 조금 추가합니다(1/4컵으로 시작하고 필요에 따라 추가). 원하는 농도가 될 때까지 조절하세요.

4. **간 맞추기**:
   - 소금과 갓 갈은 후추로 간을 맞춥니다. 다진 파슬리를 넣고 잘 섞습니다.

5. **서빙**:
   - 파스타를 접시에 담고 원한다면 추가 파슬리, 간 파르마산 치즈, 레몬 제스트로 장식합니다. 즉시 서빙하세요.

맛있게 만든 스파게티 알리오 올리오를 즐기세요!


In [59]:
translate_chain.invoke({"content":response, "language":"일본어"})

'もちろん！クラシックなパスタ料理のシンプルなレシピを紹介します：アーリオ・エ・オリオのスパゲッティ（にんにくとオリーブオイルのスパゲッティ）。\n\n### アーリオ・エ・オリオのスパゲッティレシピ\n\n#### 材料：\n\n- スパゲッティ 400g\n- にんにく 6片（薄切り）\n- エキストラバージンオリーブオイル 1/2カップ\n- 赤唐辛子フレーク 1/4小さじ（お好みで調整）\n- 塩（お好みで）\n- 黒コショウ（お好みで）\n- 新鮮なパセリ 1/4カップ（刻んだもの）\n- パルメザンチーズ（お好みで、提供時に）\n- レモンの皮（お好みで、提供時に）\n\n#### 作り方：\n\n1. **パスタを茹でる**：\n   - 大きな鍋に塩を加えた水を沸騰させます。スパゲッティを加え、パッケージの指示に従ってアルデンテになるまで茹でます。パスタの水を約1カップ取っておき、パスタを水切りします。\n\n2. **にんにくとオイルの準備**：\n   - パスタが茹でている間、大きなフライパンに中火でオリーブオイルを熱します。\n   - 薄切りにしたにんにくと赤唐辛子フレークを加え、にんにくが黄金色になり香りが立つまで、頻繁にかき混ぜながら約3～4分料理します。にんにくを焦がさないように注意してください。焦げると苦味が出ます。\n\n3. **パスタとソースを合わせる**：\n   - 水切りしたスパゲッティをにんにくとオイルのフライパンに加えます。パスタをオイルで均等にコーティングするように混ぜます。\n   - パスタが乾いているようであれば、取っておいたパスタの水を少しずつ加え（最初は1/4カップから始め、必要に応じて追加）、希望の濃度に調整します。\n\n4. **味付け**：\n   - 塩と新鮮に挽いた黒コショウで味を調えます。刻んだパセリを加え、よく混ぜます。\n\n5. **盛り付け**：\n   - パスタを皿に盛り、追加のパセリ、すりおろしたパルメザンチーズ、必要であればレモンの皮で飾ります。すぐに提供します。\n\nおいしい手作りのアーリオ・エ・オリオスパゲッティをお楽しみください！'

## Chain과 Chain간의 연결

In [76]:
# food_chain ----> translate_chain
## food_chain_prompt: 변수 - food
## translate_chain : 변수 - language, content

# RunnableParallel({"key":Runnable, "key2":Runnable})
# LCEL에서 RunnableParallel => {"key":Runnable, "key2":Runnable} |
chain = {"content":food_chain, 
         "language":RunnableLambda(lambda x : x["language"]) } | translate_chain

In [ ]:
# chain = {"content":food_chain, 
#          "language":RunnableLambda(lambda x : x["language"]) } | RunnablePassthrough()
# chain.invoke({"food":"파스타", "language":"영어"})

{'content': '## 파스타 레시피\n\n### 재료\n- 스파게티 면 200g\n- 올리브유 2큰술\n- 마늘 3쪽 (다진 것)\n- 양파 1개 (다진 것)\n- 방울토마토 200g (반으로 자른 것)\n- 시금치 100g\n- 바질 잎 적당량 (옵션)\n- 소금과 후추 (간 맞추기)\n- 파마산 치즈 (갈아서, 옵션)\n\n### 조리 도구\n- 큰 냄비\n- 팬\n- 나무 주걱\n- 체반\n\n### 조리 방법\n\n1. **면 삶기**\n   - 큰 냄비에 물을 끓입니다. 물이 끓으면 소금을 넣고 스파게티 면을 넣습니다. 포장지에 적힌 시간 동안 삶아주세요(보통 8-10분).\n   - 면이 다 삶아지면 체반에 걸러 물기를 제거하고, 약간의 올리브유를 뿌려 고루 섞어줍니다.\n\n2. **소스 준비**\n   - 팬에 올리브유를 두르고 중불에서 열을 가합니다. 다진 마늘을 넣고 향이 나기 시작할 때까지 약 1분간 볶습니다.\n   - 다진 양파를 추가하고, 양파가 투명해질 때까지 볶습니다.\n\n3. **토마토와 시금치 추가**\n   - 반으로 자른 방울토마토를 팬에 추가하고, 3-4분간 요리하여 토마토가 부드러워질 때까지 조리합니다.\n   - 시금치를 넣고 시금치가 시들해질 때까지 젓습니다.\n\n4. **면과 혼합**\n   - 삶은 스파게티 면을 팬에 넣고, 전체 재료와 잘 섞이도록 저어줍니다. 필요에 따라 소금과 후추로 간을 맞춥니다.\n   - 팬에서 불을 끈 후, 신선한 바질 잎을 위에 올려줍니다.\n\n5. **서빙**\n   - 완성된 파스타를 접시에 담고, 원한다면 갈아놓은 파마산 치즈를 뿌려 서빙합니다.\n\n### 팁\n- 다양한 재료 (예: 새우, 베이컨, 크림 소스 등)를 추가하여 맛을 변형할 수 있습니다.\n- 파스타는 항상 즉석에서 조리한 후 바로 서빙하는 것이 가장 맛있습니다. \n\n맛있게 드세요!',
 'language': '영어'}

In [77]:
res = chain.invoke({"food":"파스타", "language":"영어"})
# food -> food_chain, language -> translate_chain
# food_chain 최종결과(레시피) => {"content":레시피} -전달-> translate_chain


In [78]:
print(res)

## Basic Pasta Recipe

### Ingredients
- Pasta (spaghetti, penne, etc.) 200g
- Olive oil 2 tablespoons
- Minced garlic 3 cloves
- Onion 1 (medium size)
- Cherry tomatoes 200g
- Salt & pepper
- Basil (fresh or dried) to taste
- Parmesan cheese (optional)

### Cooking Tools
- Large pot
- Pan
- Strainer

### Cooking Method

1. **Cooking Pasta:**
   - Boil water in a large pot and add enough salt. (1 tablespoon of salt per liter of water)
   - Add the pasta to the boiling water and cook for the time indicated on the package (usually 8-12 minutes).
   - Drain the cooked pasta using a strainer, and drizzle with olive oil if desired.

2. **Preparing the Sauce:**
   - Heat olive oil in a pan over medium heat.
   - Add minced garlic and chopped onion, and sauté until the onion becomes translucent.
   - Cut the cherry tomatoes in half and add them to the pan, seasoning with salt and pepper. (Sauté for about 5-7 minutes.)

3. **Mixing:**
   - Add the cooked pasta to the pan and stir well to combi

In [79]:
chain.invoke({"food":"냉면", "language":"몽골어"})

'### Халуун ногооны жор\n\n#### Орц\n* Халуун ногооны будаа (булгасны будаа эсвэл гурилан будаа) 200г\n* Шөлийг (үхрийн шөл эсвэл загасны шөл) 1L\n* Сүү (2 том халбага)\n* Цуу (2 том халбага)\n* Элсэн чихэр (1 том халбага)\n* Үхрийн саримс (1 жижиг халбага)\n* Огурцы (1 ш)\n* Нэрс (эсвэл алим) (1/2 ш)\n* Төмс (2 ш)\n* Нори (цамцлаг)\n* Улаан перец (сонголт)\n* Сансар (сонголт)\n\n#### Шөл бэлтгэх\n1. **Шөл хийх:** Үхрийн 200г болон 1L ус тавагтаа хийж буцалгана. Буцалж эхлэхэд дунд гал дээр 30 минут орчим буцалгана.\n2. **Шөлний амт тохируулах:** Шөл ууж идэж, амт нь гүнзгий байх үед 2 том халбага сүү, 1 жижиг халбага үхрийн саримсыг нэмж дахин буцалгаад, галын тагийг хааж хөргөнө.\n\n#### Будаа бэлтгэх\n1. **Будаа буцалгах:** Тавагт ус буцалгаж, халуун ногооны будааг хийнэ. Савны зааварт заасан буцалгах хугацааг анхаарч үзүүлэхэд тохиромжтой хугацаагаар буцална.\n2. **Будаагаа хүйтэн усаар угаах:** Будаа буцалсны дараа хүйтэн усанд хэд хэдэн удаа угааж, гадаргуу нь жигд чипс мэт байх 

# 사용자 함수를 Chain에 적용하기

## 사용자 함수를 Runnable로 정의 (RunnableLambda)
- 임의의 함수를 Runnable로 정의 할 수있다.
  - chain에 포함할 기능을 함수로 정의할 때 주로 사용. 
- `RunnableLambda(함수)` 사용
  - 함수는 invoke() 메소드를 통해 입력받은 값을 받을 **한개의 파라미터**를 선언해야 한다.

## 사용자 함수를 Chain으로 정의
- Chain 을 구성하는 작업 사이에 추가 작업이 필요할 경우, 중간 결과를 모두 사용해야 하는 경우 함수로 구현한다.
- `@chain` 데코레이터를 사용해 함수에 선언한다.

### Runnable 에 사용할 **사용자 정의 함수** 구문
- 이전 Chain의 출력을 입력 받는 **파라미터를 한개** 선언한다. (첫번째 파라미터)
- `invoke()`로 호출 할때 전달 하는 추가 설정을 입력받는 파라미터를 선언한다.(두번째 파라미터 - Optional)
  - RunnableConfig 타입의 값을 받는데 Dictionary 형식으로 `{"configuable": {"설정이름":"설정값"}}` 형식으로 받는다.
- 만약 함수가 여러개의 인자를 받는 경우 단일 입력을 받아들이고 이를 여러 인수로 풀어내는 래퍼 함수를 작성하여 Runnable로 만든다.
  ```python
  def plus(num1, num2):
      ...

  def wrapper_plus(nums:dict|list):
      return plus(nums['num1'], nums['num2'])
  ```

# Cache

- 응답 결과를 저장해서 같은 질문이 들어오면 LLM에 요청하지 않고 저장된 결과를 보여주도록 한다.
    - 처리속도와 비용을 절감할 수 있다.
    - 특히 chatbot같이 비슷한 질문을 하는 경우 유용하다.
- 저장 방식은 `메모리`, `sqlite` 등 다양한 방식을 지원한다.
  
    ```python
    set_llm_cache(Cache객체)
    ```